In [2]:
from llama_index.core import PromptTemplate
import json
import torch
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from pyvi import ViTokenizer
import weaviate
from weaviate.classes.init import Auth

/home/drissdo/Desktop/VN_LAW_QA_SYSTEM/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

DATA_COLLECTION = "ND168"
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "dangvantuan/vietnamese-document-embedding"
CHUNK_SIZE = 512  # Optimized for Vietnamese text
CHUNK_OVERLAP = 50  # Small overlap to maintain context

In [4]:
# Setup vector store
# client = weaviate.connect_to_weaviate_cloud(
#     cluster_url=WEAVIATE_URL,
#     auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
# )


client = weaviate.connect_to_local(
                     host="192.168.100.125",
                        port=8080,
                        grpc_port=50051
                )
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name=DATA_COLLECTION
)

In [5]:
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model = HuggingFaceEmbedding(model_name=MODEL_NAME, device=DEVICE, trust_remote_code=True,cache_folder="/home/drissdo/.cache/huggingface/hub"),
)

In [6]:
# Create retriever
retriever = index.as_retriever(
    vector_store_query_mode="hybrid",
    similarity_top_k=20,
    alpha=0.3,
)

In [7]:
question = "Bao nhiêu tuổi mới được lái xe gắn máy "
retrieved_docs = retriever.retrieve(question)

for i, node in enumerate(retrieved_docs):
    print(f"Result {i+1}:\n")
    print(f"Text: {node.text}\n")
    print(f"Metadata: {node.metadata}\n")
    print(f"Score: {node.score}\n")
    print("="*50)

Result 1:

Text: loại phương_tiện : general mức phạt : none nội_dung vi_phạm : người đủ 16 tuổi trở lên được điều_khiển xe_gắn_máy .

Metadata: {'category': 'General', 'fine_amount': 'None', 'violation_type': 'khác', 'original_text': 'Loại phương tiện: General\nMức phạt: None\nNội dung vi phạm: Người đủ 16 tuổi trở lên được điều khiển xe gắn máy.'}

Score: 0.7591676712036133

Result 2:

Text: loại phương_tiện : general mức phạt : none nội_dung vi_phạm : người đủ 21 tuổi trở lên được cấp giấy_phép lái_xe hạng c , be .

Metadata: {'category': 'General', 'fine_amount': 'None', 'violation_type': 'khác', 'original_text': 'Loại phương tiện: General\nMức phạt: None\nNội dung vi phạm: Người đủ 21 tuổi trở lên được cấp giấy phép lái xe hạng C, BE.'}

Score: 0.5291225910186768

Result 3:

Text: loại phương_tiện : general mức phạt : none nội_dung vi_phạm : người đủ 18 tuổi trở lên được cấp giấy_phép lái_xe hạng a1 , a , b1 , b , c1 , được cấp chứng_chỉ bồi_dưỡng kiến_thức pháp_luật về giao_thông 